#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object

    We have used flask app objects as APIs from trimesters 1 and 2. It allows us to register new blueprints, which was important for us in order to create our API connecting the frontend and backend together. 

    2. SQLAlchemy db object

    We have used the db object in order to create our database from last trimester. When we set db equal to the SQLAlchemy object, we were registering our database, which allowed for the creation of our sqlite data table as well as the database containing everything that is needed.



In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User

    The class User can be thought of as creating a new person with its different characteristics. It works the same way here, as the class User is going to include multiple attributes, such as their user id, their date of birth, age, and more. 
    
    - db.Model inheritance

    The User class inherits from db.Model, which allows the class to inherit a variety of useful properties and methods, such as the ability to define database columns using db.Column and the ability to perform database operations such as adding, updating, and deleting records using methods like create(), update(), and delete().

    - _init_ method

    The __init__() method is what constructs the User object, which initializes the instance variables within the object. 
    
    - ```@property```, ```@<column>.setter```

    The @property and @setter methods are getter and setter functions, respectively, that allow the attributes to be accessed and updated.

    - create, read, update, delete methods

    The create(), read(), update(), and delete() methods all implement the CRUD (Create, Read, Update, Delete) operations for the 'users' table. The create() method adds a new user object to the table, the read() method outputs a dictionary representation of the user object, the update() method updates the attributes of the user object (includes before and after), and the delete() method removes the user object from the table.

In [3]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object

    # init method allows you to use these attributes at all times
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object

    # getter method @property, read
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    # update, create in CRUD
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object

    The "db.create_all" line creates all the tables defined in the database schema with the SQLAlchemy ORM. It reads the schema and creates the tables in the connected database.
    
    2. User Object Constructors

    The code creates instances of the User class using the constructor with different combinations of arguments, such as name, uid, password, and date of birth. It initializes these instances as u1, u2, and so on.

    3. Try / Except 

    The try/except lines create a new row in the User table with the data passed to the constructor. If the data is valid and the row is successfully created, create() will output the created User object. Otherwise, it raises an exception. The try/except block handles any exceptions that may occur during the creation process, such as duplicate user ids or other validation errors. If an exception is caught, the function prints an error message indicating the cause.


In [4]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))

        # locals include age, name, date of birth, user id, etc.
        # debugging allows you to go in to see if there is any problems with your code (ex. missing an attribute like user id)
        # once you have your users, your sqlite.db file is created 



        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

Records exist uid toby, or error.
Records exist uid niko, or error.
Records exist uid lex, or error.
Records exist uid whit, or error.
Records exist uid indi, or error.
Records exist uid raven, or error.


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by

    Looking at the code, the user.query.fliter_by is intended to distinguish a user by their user id, which allows one to know which id represents which user. 

    2. user.password

    The user.is_password checks the attributes of a user by finding and verifying the user's password. If the password found is the user's password, the boolean variable will evaluate to true. If the password found is not the user's password, the variable will be evaluated to false. 

In [5]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except

    The user.find_by_uid(uid) method is used to check if a user with the given uid already exists in the database. The try/except block is used to handle the case when no user is found with the given uid. If user.read() is called when user is None, it will raise an AttributeError, so the try/except block is used to catch this error and continue with creating a new user.

    2. user = User(...)

    user = User(...) creates a new instance of the User class with the given attributes. It initializes a new User object with the name, uid, and password provided by the user through the input() method.

    3. user.dob and try/except

    user.dob sets the date of birth attribute for the User object. The try/except block is used to handle the case when the inputted date is invalid, for example, when the inputted date does not follow the format 'YYYY-MM-DD'. If the date is invalid, the current date is used instead and the user is informed that the date was invalid.

    4. user.create() and try/except

    user.create() method is a custom method that is used to add a new row to the database table for the given User object. The try/except block is used to catch any errors that may occur when creating the new row in the database. If any errors occur, the user is informed that an unknown error has occurred with the uid provided.

In [6]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 9, 'name': 'lol', 'uid': '1', 'dob': '02-12-2012', 'age': 11}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all

    User.query.all() is a method provided by SQLAlchemy to query (retrieve data from the database) all records in the User table. It returns a list of User objects that represent all the records in the table.

    2. json_ready assignment, google List Comprehension

    The json_ready variable is assigned a list of JSON objects that represent the records in the User table. The list is generated using a list comprehension, which is an efficient way of generating lists in Python. The list comprehension [user.read() for user in table] loops through each User object in the table list and calls the read() method on each object, which returns a JSON representation of the user. The resulting list of JSON objects is then assigned to the json_ready variable.

In [7]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-14-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7, 'name': 'emu', 'uid': 'f', 'dob': '01-12-2007', 'age': 16},
 {'id': 8,
  'name': 'urmom',
  'uid': 'lolurmomandamongusmen',
  'dob': '02-19-531',
  'age': 1492},
 {'id': 9, 'name': 'lol', 'uid': '1', 'dob': '02-12-2012', 'age': 11}]

# My Own Database

In [35]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


In [36]:
import datetime

import json

from datetime import datetime

from sqlalchemy.exc import IntegrityError



class Game(db.Model):
    __tablename__ = 'games'

    # added this after getting an error that the table had already been defined
    __table_args__ = {'extend_existing': True}


    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255), unique=True, nullable=False)
    price = db.Column(db.Float, nullable=False)
    release_date = db.Column(db.Date, nullable=False)
    developer = db.Column(db.String(255), nullable=False)
    publisher = db.Column(db.String(255), nullable=False)

    def __init__(self, name, price, release_date, developer, publisher):
        self.name = name
        self.price = price
        self.release_date = release_date
        self.developer = developer
        self.publisher = publisher

    @property
    def game_info(self):
        return self.read()

    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
            return self
        except IntegrityError:
            db.session.remove()
            return None

    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "price": self.price,
            "release_date": self.release_date,
            "developer": self.developer,
            "publisher": self.publisher,
        }

    def update(self, name="", price="", release_date="", developer="", publisher=""):
        if len(name) > 0:
            self.name = name
        if len(price) > 0:
            self.price = price
        if len(release_date) > 0:
            self.release_date = release_date
        if len(developer) > 0:
            self.developer = developer
        if len(publisher) > 0:
            self.publisher = publisher
        db.session.commit()
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None


In [41]:
"""Database Creation and Testing """


# Builds working data for testing
def initGames():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        g1 = Game(name='Mario Party 9', price=169.00, release_date=datetime(2012, 2, 3), developer='NDcube', publisher='Nintendo')
        g2 = Game(name='Legend of Zelda: Tears of the Kingdom', price=69.99, release_date=datetime(2023, 5, 12), developer='Nintendo Entertaining Planning and Development', publisher='Nintendo')
 
        # locals include age, name, date of birth, user id, etc.
        # debugging allows you to go in to see if there is any problems with your code (ex. missing an attribute like user id)
        # once you have your users, your sqlite.db file is created 

        games = [g1, g2]

        """Builds sample user/note(s) data"""
        for game in games:
            try:
                '''add user to table'''
                object = game.create()
                print(f"Created new uid {object.name}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {game.name}, or error.")
                
initGames()

Records exist uid Mario Party 9, or error.
Records exist uid Legend of Zelda: Tears of the Kingdom, or error.


In [45]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_name(name):
    with app.app_context():
        game = Game.query.filter_by(name=name).first()
    return game

        
#check_credentials("indi", "123qwerty")

In [49]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    name = input("Enter the name of your game:")
    game = find_by_name(name)
    try:
        print("Found\n", game.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    price = float(input("Enter the price of your game"))
    developer = input("Enter the developer of your game")
    publisher = input("Enter the publisher of your game")
    
    
    # Initialize User object before date
    game = Game(name=name, 
                price=price, 
                developer=developer,
                publisher=publisher,
                release_date=datetime.today().date()
                )
    
    # create user.dob, fail with today as dob
    release_date = input("Enter the date your game was (or is to be) released (YYYY-MM-DD)")
    try:
        game.release_date = datetime.strptime(release_date, '%Y-%m-%d').date()
    except ValueError:
        game.release_date = datetime.today()
        print(f"Invalid date {release_date} require YYYY-mm-dd, date defaulted to {game.release_date}")
           
    # write object to database
    with app.app_context():
        try:
            object = game.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error name {name}")
        
create()

Created
 {'id': 3, 'name': 'Mario Kart Wii', 'price': 32.47, 'release_date': datetime.date(2008, 4, 10), 'developer': 'Nintendo', 'publisher': 'Nintendo'}


# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.